In [6]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np
import datetime
import time
import logging
from joblib import dump, load
from datetime import timedelta
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn import svm
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_squared_error
from sklearn.multioutput import RegressorChain
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
import xgboost as xgb
import lightgbm as lightgbm

pd.options.display.max_rows = 10


logging.basicConfig(filename='output.log', level=logging.INFO, filemode='w')

completeTimer = time.time()
typeTimer = time.time()
estimatorTimer = time.time()


# environment
isSuperComputer = True

cantonKeys = ['AG','AI','AR', 'BE', 'BL', 'BS', 'FR', 'GE', 'GL', 'GR', 'JU', 'LU', 'NE', 'NW', 'OW', 'SG', 'SH', 'SO', 'SZ', 'TG', 'TI', 'UR', 'VD', 'VS', 'ZG','ZH']

data = pd.read_csv("completedata.csv")

# last check if there are some NaN values in the data which we use
for col in data.columns:
    if data[col].isna().sum() != 0:
        print(col+" (#NaN/#NotNaN): (" + str(data[col].isna().sum())+"/"+str(data[col].notna().sum())+")")


numberOfInputWeeks = 3

#display([col for col in data.columns])


outputCategories = [
                  'case_inz_entries_7dayAverage',
                  'hosp_inz_entries_7dayAverage',
                  'death_inz_entries_7dayAverage',
                  'testPositvity',
                  'testPositvity_7dayAverageBoth',    
                  'transit_stations_percent_change_from_baseline_7dayAverage',
                  'workplaces_percent_change_from_baseline_7dayAverage',    
                   ]
numberOfOutputs = len(outputCategories)
numberOfPreComputedOutputWeeks = 4

split = numberOfOutputs * numberOfPreComputedOutputWeeks + 2

train_features = data[data['category']=='train'].iloc[:,0:-split].drop(['lastInputDay'], axis=1)
train_labels = data[data['category']=='train'].iloc[:,-split:-2]
validation1_features = data[data['category']=='validation 1'].iloc[:,0:-split].drop(['lastInputDay'], axis=1)
validation1_labels = data[data['category']=='validation 1'].iloc[:,-split:-2]
validation2_features = data[data['category']=='validation 2'].iloc[:,0:-split].drop(['lastInputDay'], axis=1)
validation2_labels = data[data['category']=='validation 2'].iloc[:,-split:-2]

'''
cId = 'AG'
train_features = data[(data['category']=='train') & (data['cantonId']==cId)].iloc[:,0:-split].drop(['lastInputDay'], axis=1)
train_labels = data[(data['category']=='train') & (data['cantonId']==cId)].iloc[:,-split:-2]
validation1_features = data[(data['category']=='validation 1') & (data['cantonId']==cId)].iloc[:,0:-split].drop(['lastInputDay'], axis=1)
validation1_labels = data[(data['category']=='validation 1') & (data['cantonId']==cId)].iloc[:,-split:-2]
validation2_features = data[(data['category']=='validation 2') & (data['cantonId']==cId)].iloc[:,0:-split].drop(['lastInputDay'], axis=1)
validation2_labels = data[(data['category']=='validation 2') & (data['cantonId']==cId)].iloc[:,-split:-2]
'''

#pip = Pipeline([('std_scaler', StandardScaler())])
pip = Pipeline([('minmax_scaler', MinMaxScaler())])

X_train = pip.fit_transform(train_features[train_features.columns].values)
X_valid1 = pip.transform(validation1_features[train_features.columns].values)
X_valid2 = pip.transform(validation2_features[train_features.columns].values)


In [2]:
# numberOfOutputWeeks = 1  week number which will be computed, possibilities 1,2,3 or 4
modelId = 0
results = pd.DataFrame(columns=['model',
                                'model type',
                                'target',
                                'week',
                                'model rsme 1', 
                                'model rsme 2'])
results.to_csv("results.csv", index=False)
for numberOfOutputWeeks in [1,2,3,4]:
    logging.info("============== fitting week "+str(numberOfOutputWeeks)+" ============== ")
    
    # (str(type(estimator)) == "<class 'tensorflow.python.keras.engine.functional.Functional'>")



    # --------------------------------------------------------------------------------------------------------

    typeTimer = time.time()
    logging.info("============== fitting normal sklearn estimators ============== ")

    # fitting normal sklearn estimators
    
    

    estimators = [
        linear_model.Ridge(alpha=1),
        linear_model.Lasso(alpha=1),
    ]


    if isSuperComputer:
        # all all estimators which we want to search through
        estimators = []
        
        kernels = ['linear','poly','polynomial','rbf','laplacian','sigmoid','cosine']
        for alpha in [0.01,0.1,10]:
            for k in kernels:
                estimators.append(KernelRidge(kernel=k, alpha=alpha))

        # add linear models
        estimators.append(linear_model.Ridge(alpha=100))
        estimators.append(linear_model.Ridge(alpha=1000))
        
        '''
        estimators.append(linear_model.Ridge(alpha=0))
        estimators.append(linear_model.Lasso(alpha=0)) 
        
        for alpha in [0.00001, 0.0001,0.001,0.01,0.1,1,10]:
            estimators.append(linear_model.Ridge(alpha=alpha))
            estimators.append(linear_model.Lasso(alpha=alpha)) 
            for lr in ['invscaling']: #,'adaptive','optimal'
                for p in ['l1','l2']: #,'elasticnet'
                    estimators.append(linear_model.SGDRegressor(learning_rate = lr,penalty=p, alpha=alpha,max_iter=10000, tol=0.00001, n_iter_no_change=10))

        # add SVRs
        for c in [0.001,0.01,0.1,1,10, 100]: 
            estimators.append(svm.SVR(C=c)) 


        # add MLPs
        for alpha in [0.00001, 0.0001,0.001,0.01,0.1,1,10]: 
            for layer in [
                (20),
                (100),
                (20,20),
                (100,10),
                (100,50,10),
                (100,100,10),
            ]:
                estimators.append(MLPRegressor(hidden_layer_sizes=layer, alpha=alpha, max_iter=1000))


        # add LGBMs & XGBoost
        for regalpha in [0.01, 0.1,0,1,10]:
            for reglambda in [0.01, 0.1,0,1,10]: 
                estimators.append(lightgbm.LGBMRegressor(n_estimators=1000, reg_alpha=regalpha, reg_lambda=reglambda, n_jobs=-1))
                estimators.append(xgb.XGBRegressor(n_estimators=1000, reg_alpha=regalpha, reg_lambda=reglambda, n_jobs=-1))  

        # add random forrest  
        for nrOfEstimators in [100,200,500]: #1000
            for maxdepth in [None, 2, 4]:
                for mss in [2,3,4]:
                    for msl in [1,2,3]:
                        estimators.append(RandomForestRegressor(n_estimators=nrOfEstimators, 
                                                                max_depth=maxdepth, 
                                                                min_samples_split=mss, 
                                                                min_samples_leaf=msl,
                                                                n_jobs=-1))
        '''
    logging.info("number of estimators to fit: " + str(len(estimators)*len(outputCategories))+" ")


    while len(estimators) > 0:
        estimator = estimators.pop(0)
        estimatorTimer = time.time()
        logging.info(str(estimator)+"")
        for category in outputCategories:
            logging.info("-- " + category+"")

            trainOutput = train_labels[["output_"+category+"_"+str(numberOfOutputWeeks-1)]] 
            reg = estimator.fit(X_train, trainOutput.values)           
            # save the fitted regressor
            #dump(reg, 'models/'+str(modelId)+'.joblib') 
            #reg = load('models/'+str(modelId)+'.joblib') 
            modelId = modelId + 1

            predictions_valid1 = pd.DataFrame(reg.predict(X_valid1), index=validation1_labels.index, columns=["pred_week_" + str(numberOfOutputWeeks-1)+"_"+category]) 
            predictions_valid2 = pd.DataFrame(reg.predict(X_valid2), index=validation2_labels.index, columns=["pred_week_" + str(numberOfOutputWeeks-1)+"_"+category])

            valid1Output = validation1_labels[["output_"+category+"_"+str(numberOfOutputWeeks-1)]]
            valid2Output = validation2_labels[["output_"+category+"_"+str(numberOfOutputWeeks-1)]]

            rmse1 = np.sqrt(mean_squared_error(predictions_valid1["pred_week_" + str(numberOfOutputWeeks-1)+"_"+category], valid1Output.values))
            rmse2 = np.sqrt(mean_squared_error(predictions_valid2["pred_week_" + str(numberOfOutputWeeks-1)+"_"+category], valid2Output.values))

            d = {'model':[reg],'model type': ["sklearn_single_output"],'target':[category],'week':[numberOfOutputWeeks-1], 'model rsme 1':[rmse1], 'model rsme 2':[rmse2]}
            df = pd.DataFrame(data=d)
            df.to_csv("results.csv",mode='a', header=False, index=False)

        logging.info(str(estimator)+" (%s) " % (time.time() - estimatorTimer))

    logging.info("============== finished fitting normal sklearn estimators ============== (%s) " % (time.time() - typeTimer))           

    # ------------------------------------------------------------------------------------------------                       

    typeTimer = time.time()
    logging.info("============== fitting single output keras estimators ============== ")
    start = time.time()
    # fitting single output keras estimators

    numberOfOutputsForMultiOutput = len(outputCategories)
    epochs = 1000
    batch_size = 32 #16

    def getNormalKerasModel(name, learningrate, nrOfNeurons, isMultiOutput):
        input_ = keras.layers.Input(shape=X_train.shape[1:])
        hidden1 = keras.layers.Dense(nrOfNeurons, activation="relu")(input_)
        output = keras.layers.Dense(numberOfOutputsForMultiOutput)(hidden1) if isMultiOutput else keras.layers.Dense(1)(hidden1)
        model = keras.Model(name=name, inputs=[input_],outputs=[output])
        model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=learningrate))
        return model


    def getDropoutKerasModel(name, learningrate, nrOfNeurons, isMultiOutput):
        input_ = keras.layers.Input(shape=X_train.shape[1:])
        hidden1 = keras.layers.Dense(nrOfNeurons, activation="relu")(input_)
        dropout1 = keras.layers.Dropout(rate=0.5)(hidden1)
        output = keras.layers.Dense(numberOfOutputsForMultiOutput)(dropout1) if isMultiOutput else keras.layers.Dense(1)(dropout1)
        model = keras.Model(name=name, inputs=[input_],outputs=[output])
        model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=learningrate))
        return model

    def getRegularizedKerasModel(name, learningrate, nrOfNeurons, alpha, isMultiOutput):
        input_ = keras.layers.Input(shape=X_train.shape[1:])
        hidden1 = keras.layers.Dense(nrOfNeurons, activation="relu", kernel_regularizer=keras.regularizers.l2(alpha))(input_)
        output = keras.layers.Dense(numberOfOutputsForMultiOutput)(hidden1) if isMultiOutput else keras.layers.Dense(1)(hidden1)
        model = keras.Model(name=name, inputs=[input_],outputs=[output])
        model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=learningrate))
        return model

    def getTwoWayKerasModel(name, learningrate, nrOfNeurons, isMultiOutput):
        input_ = keras.layers.Input(shape=X_train.shape[1:])
        hidden1 = keras.layers.Dense(nrOfNeurons, activation="relu")(input_)
        concat = keras.layers.Concatenate()([input_,hidden1])
        output = keras.layers.Dense(numberOfOutputsForMultiOutput)(concat) if isMultiOutput else keras.layers.Dense(1)(concat)
        model = keras.Model(name=name, inputs=[input_],outputs=[output])
        model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=learningrate))
        return model

    def getComplexKerasModel(name, learningrate, nrOfNeurons1, nrOfNeurons2, isMultiOutput):
        input_ = keras.layers.Input(shape=X_train.shape[1:])
        hidden1 = keras.layers.Dense(nrOfNeurons1, activation="relu")(input_)
        hidden2 = keras.layers.Dense(nrOfNeurons2, activation="relu")(hidden1)
        output = keras.layers.Dense(numberOfOutputsForMultiOutput)(hidden1) if isMultiOutput else keras.layers.Dense(1)(hidden2)
        model = keras.Model(name=name, inputs=[input_],outputs=[output])
        model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=learningrate))
        return model

    def getTwoWayComplexDropoutKerasModel(name, learningrate, nrOfNeurons1, nrOfNeurons2, nrOfNeurons3, nrOfNeurons4, nrOfNeurons5, isMultiOutput):
        input_ = keras.layers.Input(shape=X_train.shape[1:])
        hidden1 = keras.layers.Dense(nrOfNeurons1, activation="relu")(input_)
        dropout1 = keras.layers.Dropout(rate=0.5)(hidden1)
        hidden2 = keras.layers.Dense(nrOfNeurons2, activation="relu")(hidden1)
        dropout1 = keras.layers.Dropout(rate=0.5)(hidden2)
        hidden3 = keras.layers.Dense(nrOfNeurons3, activation="relu")(hidden2)
        dropout1 = keras.layers.Dropout(rate=0.5)(hidden3)
        hidden4 = keras.layers.Dense(nrOfNeurons4, activation="relu")(hidden3)
        dropout1 = keras.layers.Dropout(rate=0.5)(hidden4)
        hidden5 = keras.layers.Dense(nrOfNeurons5, activation="relu")(hidden4)
        dropout1 = keras.layers.Dropout(rate=0.5)(hidden5)
        concat = keras.layers.Concatenate()([input_,dropout1])
        output = keras.layers.Dense(numberOfOutputsForMultiOutput)(concat) if isMultiOutput else keras.layers.Dense(1)(concat)
        model = keras.Model(name=name, inputs=[input_],outputs=[output])
        model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=learningrate))
        return model
    
    def getDeepDropoutKerasModel(name, learningrate, nrOfNeurons, isMultiOutput):
        input_ = keras.layers.Input(shape=X_train.shape[1:])
        dropout0 = keras.layers.Dropout(rate=0.3)(input_)
        hidden1 = keras.layers.Dense(nrOfNeurons, activation="relu")(dropout0)
        dropout1 = keras.layers.Dropout(rate=0.3)(hidden1)
        hidden2 = keras.layers.Dense(nrOfNeurons, activation="relu")(dropout1)
        dropout2 = keras.layers.Dropout(rate=0.3)(hidden2)
        hidden3 = keras.layers.Dense(nrOfNeurons, activation="relu")(dropout2)
        dropout3 = keras.layers.Dropout(rate=0.3)(hidden3)
        hidden4 = keras.layers.Dense(nrOfNeurons, activation="relu")(dropout3)
        dropout4 = keras.layers.Dropout(rate=0.3)(hidden4)
        hidden5 = keras.layers.Dense(nrOfNeurons, activation="relu")(dropout4)
        dropout5 = keras.layers.Dropout(rate=0.3)(hidden5)
        hidden6 = keras.layers.Dense(nrOfNeurons, activation="relu")(dropout5)
        dropout6 = keras.layers.Dropout(rate=0.3)(hidden6)
        hidden7 = keras.layers.Dense(nrOfNeurons, activation="relu")(dropout6)
        dropout7 = keras.layers.Dropout(rate=0.3)(hidden7)
        hidden8 = keras.layers.Dense(nrOfNeurons, activation="relu")(dropout7)
        dropout8 = keras.layers.Dropout(rate=0.3)(hidden8)
        output = keras.layers.Dense(numberOfOutputsForMultiOutput)(dropout8) if isMultiOutput else keras.layers.Dense(1)(dropout8)
        model = keras.Model(name=name, inputs=[input_],outputs=[output])
        model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=learningrate))
        return model


    estimators = []

    estimators.append(getNormalKerasModel("simple", 0.0001, 10, False))

    if isSuperComputer:
        # all all estimators which we want to search through
        estimators = []
        
        '''
        for learningrate in [0.001,0.0001,0.00001]:
            estimators.append(getNormalKerasModel("normal_learningrate_"+str(learningrate)+"_batchsize_"+str(batch_size), learningrate, 100, False))
            estimators.append(getDropoutKerasModel("dropout_learningrate_"+str(learningrate)+"_batchsize_"+str(batch_size), learningrate, 100, False))
            estimators.append(getTwoWayKerasModel("twoway_learningrate_"+str(learningrate)+"_batchsize_"+str(batch_size), learningrate, 100, False))
            estimators.append(getComplexKerasModel("complex_learningrate_"+str(learningrate)+"_batchsize_"+str(batch_size), learningrate, 100, 10, False))
            estimators.append(getTwoWayComplexDropoutKerasModel("twowaycomplexdropout_learningrate_"+str(learningrate)+"_batchsize_"+str(batch_size), learningrate, 200, 150, 100, 50, 10, False))

            for alpha in [0.0001, 0.001, 0.01, 0.1]: 
                estimators.append(getRegularizedKerasModel("regularized_learningrate_"+str(learningrate)+"_alpha_"+str(alpha)+"_batchsize_"+str(batch_size), learningrate, 100, alpha, False))     
        '''
        for learningrate in [0.001,0.0001,0.00001]:
            estimators.append(getDeepDropoutKerasModel("DeepDropout"+str(learningrate)+"_batchsize_"+str(batch_size)+"_neuronr_"+str(100), learningrate, 100, False))
            estimators.append(getNormalKerasModel("normal_learningrate_"+str(learningrate)+"_batchsize_"+str(batch_size)+"_neuronr_"+str(200), learningrate, 200, False))
            estimators.append(getDropoutKerasModel("dropout_learningrate_"+str(learningrate)+"_batchsize_"+str(batch_size)+"_neuronr_"+str(200), learningrate, 200, False))
            estimators.append(getTwoWayKerasModel("twoway_learningrate_"+str(learningrate)+"_batchsize_"+str(batch_size)+"_neuronr_"+str(200), learningrate, 200, False))
            estimators.append(getComplexKerasModel("complex_learningrate_"+str(learningrate)+"_batchsize_"+str(batch_size)+"_neuronr_"+str(200)+"_"+str(20), learningrate, 200, 20, False))
            estimators.append(getTwoWayComplexDropoutKerasModel("twowaycomplexdropout_learningrate_"+str(learningrate)+"_batchsize_"+str(batch_size)+"_neuronr_"+str(300)+"_"+str(150)+"_"+str(100)+"_"+str(50)+"_"+str(20), learningrate, 300, 150, 100, 50, 20, False))

            for alpha in [0.0001, 0.001, 0.01, 0.1]: 
                estimators.append(getRegularizedKerasModel("regularized_learningrate_"+str(learningrate)+"_alpha_"+str(alpha)+"_batchsize_"+str(batch_size)+"_neuronr_"+str(200), learningrate, 200, alpha, False)) 


    logging.info("number of estimators to fit: " + str(len(estimators)*len(outputCategories)))

    while len(estimators) > 0:
        estimator = estimators.pop(0)
        estimatorTimer = time.time()

        logging.info(str(estimator.name))
        for category in outputCategories:

            logging.info("-- " + category)

            y_train = train_labels[["output_"+category+"_"+str(numberOfOutputWeeks-1)]].values 
            y_valid1 = validation1_labels[["output_"+category+"_"+str(numberOfOutputWeeks-1)]].values 
            estimator.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=0, validation_data=(X_valid1,y_valid1), callbacks=[keras.callbacks.EarlyStopping(patience=10)])
            reg = estimator

            # save the fitted regressor
            #reg.save('models/'+str(modelId))
            #reg = keras.models.load_model('models/'+str(modelId))
            modelId = modelId + 1

            predictions_valid1 = pd.DataFrame(reg.predict(X_valid1), index=validation1_labels.index, columns=["pred_week_" + str(numberOfOutputWeeks-1)+"_"+category]) 
            predictions_valid2 = pd.DataFrame(reg.predict(X_valid2), index=validation2_labels.index, columns=["pred_week_" + str(numberOfOutputWeeks-1)+"_"+category])

            valid1Output = validation1_labels[["output_"+category+"_"+str(numberOfOutputWeeks-1)]]
            valid2Output = validation2_labels[["output_"+category+"_"+str(numberOfOutputWeeks-1)]]


            rmse1 = np.sqrt(mean_squared_error(predictions_valid1["pred_week_" + str(numberOfOutputWeeks-1)+"_"+category], valid1Output.values))
            rmse2 = np.sqrt(mean_squared_error(predictions_valid2["pred_week_" + str(numberOfOutputWeeks-1)+"_"+category], valid2Output.values))

            d = {'model':[reg.name],'model type': ["keras_single_output"],'target':[category],'week':[numberOfOutputWeeks-1], 'model rsme 1':[rmse1], 'model rsme 2':[rmse2]}
            df = pd.DataFrame(data=d)
            df.to_csv("results.csv",mode='a', header=False, index=False)

        logging.info(str(estimator.name)+" (%s)" % (time.time() - estimatorTimer))

    logging.info("============== finished fitting single output keras estimators ============== (%s) " % (time.time() - typeTimer))   

    # ----------------------------------------------------------------------------------------------------

    typeTimer = time.time()
    logging.info("============== fitting multi output keras estimators ============== ")

    estimators = []

    estimators.append(getNormalKerasModel("simple", 0.0001, 10, True))

    if isSuperComputer:
        # all all estimators which we want to search through
        estimators = []
        '''
        for learningrate in [0.001,0.0001,0.00001]:
            estimators.append(getNormalKerasModel("normal_learningrate_"+str(learningrate), learningrate, 100, True))
            estimators.append(getDropoutKerasModel("dropout_learningrate_"+str(learningrate), learningrate, 100, True))
            estimators.append(getTwoWayKerasModel("twoway_learningrate_"+str(learningrate), learningrate, 100, True))
            estimators.append(getComplexKerasModel("complex_learningrate_"+str(learningrate), learningrate, 100, 10, True))
            estimators.append(getTwoWayComplexDropoutKerasModel("twowaycomplexdropout_learningrate_"+str(learningrate)+"_batchsize_"+str(batch_size), learningrate, 200, 150, 100, 50, 10, True))

            for alpha in [0.01, 0.001, 0.0001]: #0.1
                estimators.append(getRegularizedKerasModel("regularized_learningrate_"+str(learningrate)+"_alpha_"+str(alpha)+"_batchsize_"+str(batch_size), learningrate, 100, alpha, True))     

        '''
        for learningrate in [0.001,0.0001,0.00001]:
                estimators.append(getDeepDropoutKerasModel("DeepDropout"+str(learningrate)+"_batchsize_"+str(batch_size)+"_neuronr_"+str(100), learningrate, 100, True))
                estimators.append(getNormalKerasModel("normal_learningrate_"+str(learningrate)+"_batchsize_"+str(batch_size)+"_neuronr_"+str(200), learningrate, 200, True))
                estimators.append(getDropoutKerasModel("dropout_learningrate_"+str(learningrate)+"_batchsize_"+str(batch_size)+"_neuronr_"+str(200), learningrate, 200, True))
                estimators.append(getTwoWayKerasModel("twoway_learningrate_"+str(learningrate)+"_batchsize_"+str(batch_size)+"_neuronr_"+str(200), learningrate, 200, True))
                estimators.append(getComplexKerasModel("complex_learningrate_"+str(learningrate)+"_batchsize_"+str(batch_size)+"_neuronr_"+str(200)+"_"+str(20), learningrate, 200, 20, True))
                estimators.append(getTwoWayComplexDropoutKerasModel("twowaycomplexdropout_learningrate_"+str(learningrate)+"_batchsize_"+str(batch_size)+"_neuronr_"+str(300)+"_"+str(150)+"_"+str(100)+"_"+str(50)+"_"+str(20), learningrate, 300, 150, 100, 50, 20, True))

                for alpha in [0.0001, 0.001, 0.01, 0.1]: 
                    estimators.append(getRegularizedKerasModel("regularized_learningrate_"+str(learningrate)+"_alpha_"+str(alpha)+"_batchsize_"+str(batch_size)+"_neuronr_"+str(200), learningrate, 200, alpha, True))             

    
    logging.info("number of estimators to fit: " + str(len(estimators)))

    start = time.time()

    while len(estimators) > 0:
        estimator = estimators.pop(0)
        estimatorTimer = time.time()
        logging.info(str(estimator.name))
        logging.info("-- all categories")


        y_train = train_labels[["output_"+category+"_"+str(numberOfOutputWeeks-1) for category in outputCategories]].values
        y_valid1 = validation1_labels[["output_"+category+"_"+str(numberOfOutputWeeks-1) for category in outputCategories]].values 
        estimator.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=0, validation_data=(X_valid1,y_valid1), callbacks=[keras.callbacks.EarlyStopping(patience=10)])
        reg = estimator

        # save the fitted regressor
        #reg.save('models/'+str(modelId))
        #reg = keras.models.load_model('models/'+str(modelId))
        modelId = modelId + 1

        predictions_valid1 = pd.DataFrame(reg.predict(X_valid1), index=validation1_labels.index, columns=["pred_week_" + str(numberOfOutputWeeks-1)+"_"+category for category in outputCategories]) 
        predictions_valid2 = pd.DataFrame(reg.predict(X_valid2), index=validation2_labels.index, columns=["pred_week_" + str(numberOfOutputWeeks-1)+"_"+category for category in outputCategories])

        valid1Output = validation1_labels[["output_"+category+"_"+str(numberOfOutputWeeks-1) for category in outputCategories]]
        valid2Output = validation2_labels[["output_"+category+"_"+str(numberOfOutputWeeks-1) for category in outputCategories]]

        for category in outputCategories:
            rmse1 = np.sqrt(mean_squared_error(predictions_valid1["pred_week_" + str(numberOfOutputWeeks-1)+"_"+category], valid1Output["output_"+category+"_"+str(numberOfOutputWeeks-1)]))
            rmse2 = np.sqrt(mean_squared_error(predictions_valid2["pred_week_" + str(numberOfOutputWeeks-1)+"_"+category], valid2Output["output_"+category+"_"+str(numberOfOutputWeeks-1)]))

            d = {'model':[reg.name],'model type': ["keras_multi_output"],'target':[category],'week':[numberOfOutputWeeks-1], 'model rsme 1':[rmse1], 'model rsme 2':[rmse2]}
            df = pd.DataFrame(data=d)
            df.to_csv("results.csv",mode='a', header=False, index=False)

        logging.info(str(estimator.name)+" (%s)" % (time.time() - estimatorTimer))

    logging.info("============== finished fitting multi output keras estimators ============== (%s) " % (time.time() - typeTimer))

    # ----------------------------------------------------------------------------------------------------
    typeTimer = time.time()

    logging.info("============== fitting multioutput sklearn estimators ============== ")
    # fitting multioutput sklearn estimators
    estimators = [
        linear_model.MultiTaskLasso(),
    ]
    if isSuperComputer:
        # all all estimators which we want to search through

        estimators = []
        
        estimators.append(linear_model.Ridge(alpha=100))
        estimators.append(linear_model.Ridge(alpha=1000))

        '''
        kernels = ['linear','poly','polynomial','rbf','laplacian','sigmoid','cosine']
        for alpha in [0.01,0.1,10]:
            for k in kernels:
                estimators.append(KernelRidge(kernel=k, alpha=alpha))

        for alpha in [0.00001, 0.0001,0.001,0.01,0.1,1,10]: 
            estimators.append(linear_model.MultiTaskLasso(alpha=alpha))
        
        for alpha in [0.00001, 0.0001,0.001,0.01,0.1,0,1,10]: 
            estimators.append(linear_model.Lasso(alpha=alpha))
            estimators.append(linear_model.Ridge(alpha=alpha))

        for alpha in [0.00001, 0.0001,0.001,0.01,0.1,1,10]:
            for layer in [
                (20),
                (100),
                (20,20),
                (100,10),
                (100,50,10),
                (100,100,10),
            ]:
                estimators.append(MLPRegressor(hidden_layer_sizes=layer, alpha=alpha, max_iter=1000))

        for nrOfEstimators in [100,200,500]: #1000
            for maxdepth in [None, 2, 4]:
                for mss in [2,3,4]:
                    for msl in [1,2,3]:
                        estimators.append(RandomForestRegressor(n_estimators=nrOfEstimators, 
                                                                max_depth=maxdepth, 
                                                                min_samples_split=mss, 
                                                                min_samples_leaf=msl,
                                                                n_jobs=-1))

        '''
    logging.info("number of estimators to fit: " + str(len(estimators))+" ")

    while len(estimators) > 0:
        estimator = estimators.pop(0)
        estimatorTimer = time.time()
        logging.info(str(estimator))
        logging.info("-- all categories")


        trainOutput = train_labels[["output_"+category+"_"+str(numberOfOutputWeeks-1) for category in outputCategories]]
        reg = estimator.fit(X_train, trainOutput.values)
        # save the fitted regressor
        #dump(reg, 'models/'+str(modelId)+'.joblib') 
        #reg = load('models/'+str(modelId)+'.joblib') 
        modelId = modelId + 1

        predictions_valid1 = pd.DataFrame(reg.predict(X_valid1), index=validation1_labels.index, columns=["pred_week_" + str(numberOfOutputWeeks-1)+"_"+category for category in outputCategories]) 
        predictions_valid2 = pd.DataFrame(reg.predict(X_valid2), index=validation2_labels.index, columns=["pred_week_" + str(numberOfOutputWeeks-1)+"_"+category for category in outputCategories])

        valid1Output = validation1_labels[["output_"+category+"_"+str(numberOfOutputWeeks-1) for category in outputCategories]]
        valid2Output = validation2_labels[["output_"+category+"_"+str(numberOfOutputWeeks-1) for category in outputCategories]]

        for category in outputCategories:
            rmse1 = np.sqrt(mean_squared_error(predictions_valid1["pred_week_" + str(numberOfOutputWeeks-1)+"_"+category], valid1Output["output_"+category+"_"+str(numberOfOutputWeeks-1)]))
            rmse2 = np.sqrt(mean_squared_error(predictions_valid2["pred_week_" + str(numberOfOutputWeeks-1)+"_"+category], valid2Output["output_"+category+"_"+str(numberOfOutputWeeks-1)]))

            d = {'model':[reg],'model type': ["sklearn_multi_output"],'target':[category],'week':[numberOfOutputWeeks-1], 'model rsme 1':[rmse1], 'model rsme 2':[rmse2]}
            df = pd.DataFrame(data=d)
            df.to_csv("results.csv",mode='a', header=False, index=False)

        logging.info(str(estimator)+" (%s) " % (time.time() - estimatorTimer))

    logging.info("============== finished fitting multioutput sklearn estimators ============== (%s) " % (time.time() - typeTimer))

    # -----------------------------------------------------------------------------------------------------


    logging.info("============== finished ============== %s " % (time.time() - completeTimer))